In [77]:
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
import sklearn.preprocessing
import sklearn.decomposition
from sklearn.linear_model import LogisticRegressionCV
from sklearn.datasets import fetch_mldata
from helpers import batch_iter

In [78]:
DATA_HOME = "./data"
np.random.seed(42)

In [79]:
# Load MNIST Data
mnist = fetch_mldata('MNIST original', data_home=DATA_HOME)
data_x = sklearn.preprocessing.scale(mnist.data)
x_train, x_test, y_train, y_test = train_test_split(data_x, mnist.target, test_size=0.1, random_state=42)

/Users/dennybritz/projects/venvs/tensorflow/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
class Autoencoder:
    
    def build_layer(self, output_dim, prev_layer, activation_func=tf.tanh):
        """
        Builds a single hidden layer.
        """
        input_dim = prev_layer.get_shape().as_list()[1]
        W_init = tf.random_uniform([input_dim, output_dim], -1.0/np.sqrt(input_dim), 1.0/np.sqrt(input_dim))
        W = tf.Variable(W_init, name="W")
        b = tf.Variable(tf.zeros([output_dim]), name="b")
        return activation_func(tf.nn.xw_plus_b(prev_layer, W, b))
    
    def __init__(self, x, hidden_dims=[32]):        
        # Keeps track of the hidden layers so we can refer to them later
        self.hidden_layers = []
        prev_layer = x
        
        # For each dimension, build a hidden layer
        for i, layer in enumerate(hidden_dims):
            with tf.variable_scope("hidden-{}".format(i)):
                prev_layer = self.build_layer(hidden_dims[i], prev_layer)
                self.hidden_layers.append(prev_layer)
               
        # Build output (reconstruction) layer
        with tf.variable_scope("output"):
            output_dim = x.get_shape().as_list()[1]
            self.output = self.build_layer(output_dim, prev_layer)
        
        # Squared loss function
        self.total_loss = tf.reduce_sum(tf.square(x - self.output))
        self.avg_loss = tf.reduce_mean(tf.square(x - self.output))

In [ ]:
BATCH_SIZE = 32
NUM_EPOCHS = 30
PRINT_LOSS_EVERY=2000
LAYERS = [32, 32]

graph = tf.Graph()
sess = tf.Session(graph=graph)

with graph.as_default(), sess.as_default():
    x = tf.placeholder(tf.float32, [None, x_train.shape[1]])
    ae = Autoencoder(x, LAYERS)
    
    # Optimization
    global_step = tf.Variable(0, name="global_step", trainable=False)
    optimizer = tf.train.AdamOptimizer(1e-4)
    train_op = optimizer.minimize(ae.total_loss, global_step=global_step)
    
    # Initialize variables
    sess.run(tf.initialize_all_variables())
    
    batches = batch_iter(x_train, BATCH_SIZE, NUM_EPOCHS)
    # For each batch...
    for x_batch in batches:
        feed_dict = { x: x_batch }
        _, loss, step = sess.run([train_op, ae.avg_loss, global_step], feed_dict)
        if step % PRINT_LOSS_EVERY == 0:
            total_loss =  sess.run(ae.avg_loss, { x: x_train })
            print("{}: Mean Loss: {:g}".format(step ,total_loss))
    print("{}: Final Mean Loss: {:g}".format(step ,total_loss))

In [ ]:
# Get the compressed representation of the input X
with graph.as_default(), sess.as_default():
    x_train_transformed = sess.run(ae.hidden_layers[1], { x: x_train })
    x_test_transformed = sess.run(ae.hidden_layers[1], { x: x_test })

In [ ]:
clf = KNeighborsClassifier()
clf.fit(x_train_transformed, y_train)
y_pred = clf.predict(x_test_transformed)
print(classification_report(y_test, y_pred, digits=3))